# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [22]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../Starter_Code/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,grytviken,-54.2811,-36.5092,0.37,97,100,3.34,GS,1728617081
1,1,tuba city,36.1350,-111.2399,22.68,17,69,1.37,US,1728617083
2,2,bandundu,-3.3167,17.3667,22.20,95,25,0.78,CD,1728617084
3,3,kemijarvi,66.7131,27.4306,4.59,95,100,7.08,FI,1728617085
4,4,albany,42.6001,-73.9662,8.58,67,5,0.89,US,1728617086


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [12]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng', 'Lat',
    geo=True,
    tiles='OSM',
    size='Humidity',
    scale=0.1,
    color='City',
    hover_cols=['City', 'Humidity']
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [16]:
# Narrow down cities that fit criteria and drop any results with null values
temperature_threshold = 25 
humidity_threshold = 50
wind_speed_threshold = 4.5
cloudiness_threshold = 0

filtered_cities = city_data_df[
    (city_data_df['Max Temp'] >= temperature_threshold) & 
    (city_data_df['Humidity'] <= humidity_threshold)
]
# Drop any rows with null values
filtered_cities = filtered_cities.dropna()

# Display sample data
filtered_cities.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,8,kidal,18.4411,1.4078,30.51,18,41,1.70,ML,1728617090
34,34,geraldton,-28.7667,114.6000,30.23,42,0,4.63,AU,1728617120
50,50,kone,-21.0595,164.8658,29.46,41,0,4.47,NC,1728617139
73,73,el matama,16.7095,33.3565,30.22,47,8,7.12,SD,1728617165
77,77,itaueira,-7.6033,-43.0256,28.69,37,8,1.74,BR,1728617170


### Step 3: Create a new DataFrame called `hotel_df`.

In [17]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_cities[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()


,City,Country,Lat,Lng,Humidity,Hotel Name
8,kidal,ML,18.4411,1.4078,18,
34,geraldton,AU,-28.7667,114.6000,42,
50,kone,NC,-21.0595,164.8658,41,
73,el matama,SD,16.7095,33.3565,47,
77,itaueira,BR,-7.6033,-43.0256,37,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [20]:
# Set parameters to search for a hotel
radius = 10000  
params = {
    'type': 'lodging',
    'radius': radius,
    'apiKey': geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat};{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kidal - nearest hotel: No hotel found
geraldton - nearest hotel: No hotel found
kone - nearest hotel: No hotel found
el matama - nearest hotel: No hotel found
itaueira - nearest hotel: No hotel found
illizi - nearest hotel: No hotel found
goundam - nearest hotel: No hotel found
crane - nearest hotel: No hotel found
mareeba - nearest hotel: No hotel found
codajas - nearest hotel: No hotel found
thakhek - nearest hotel: No hotel found
banjar - nearest hotel: No hotel found
anori - nearest hotel: No hotel found
taro - nearest hotel: No hotel found
taoudenni - nearest hotel: No hotel found
aoulef - nearest hotel: No hotel found
viqueque - nearest hotel: No hotel found
novo aripuana - nearest hotel: No hotel found
alice springs - nearest hotel: No hotel found
charters towers - nearest hotel: No hotel found
anar darah - nearest hotel: No hotel found
sur - nearest hotel: No hotel found
as sulayyil - nearest hotel: No hotel found
dalbandin - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
8,kidal,ML,18.4411,1.4078,18,No hotel found
34,geraldton,AU,-28.7667,114.6000,42,No hotel found
50,kone,NC,-21.0595,164.8658,41,No hotel found
73,el matama,SD,16.7095,33.3565,47,No hotel found
77,itaueira,BR,-7.6033,-43.0256,37,No hotel found
179,illizi,DZ,26.4833,8.4667,25,No hotel found
210,goundam,ML,16.4145,-3.6708,26,No hotel found
215,crane,US,31.3974,-102.3501,25,No hotel found
230,mareeba,AU,-17.0000,145.4333,47,No hotel found
239,codajas,BR,-3.8367,-62.0569,41,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [23]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    'Lng', 'Lat',
    geo=True,
    tiles='OSM',  
    size='Humidity',  
    scale=0.1,  
    color='City',  
    hover_cols=['City', 'Hotel Name', 'Country', 'Humidity']  
)

# Display the map
map_plot.opts(tools=['hover'])


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)